In [2]:
import os, json, sys, pandas as pd, numpy as np, re

### Processor

In [40]:
with open('./data/raw/amd_socket.json') as f:
    amd_socket = json.load(f)

with open('./data/raw/intel_socket.json') as f:
    intel_socket = json.load(f)

list_socket = amd_socket['result'] + intel_socket['result']

In [35]:



with open('./data/raw/prosesor.json') as f:
    processors = json.load(f)

list_processors = processors['result']
#remove processor if processor["PLINK"] contains bundle
list_processors = [processor for processor in list_processors if "bundle" not in processor["PLINK"].lower()]

processed_processor = []
for socket in list_socket:
    socket_processors = [processor for processor in list_processors if socket["PLINK"].upper() in processor["PLINK"].upper()]
    for processor in socket_processors:
        processor["socket"] = socket["PLINK"]
        processor['brand'] = socket["BNAME"]
    processed_processor += socket_processors
print([processor for processor in processed_processor if processor['socket'] == 'FM2'][0]['PPRCZ'])

for processor in processed_processor:
    processor['price'] = processor['PPRCZ'][0] if processor.get('PPRCZ') is not None else None
    processor['image'] = processor['PIMGZ'][0] if processor.get('PIMGZ') is not None else None
    #for deleted_key in ['PPRCZ', 'PIMGZ', 'PSTTS', 'PDTLS', 'PBEST', 'RTYPE']:
    #    if processor.get(deleted_key) is not None:
    #        del processor[deleted_key]

#remove duplicated processor
processed_processor = list({v['PCODE']:v for v in processed_processor}.values())


for processor in processed_processor:
    for deleted_key in ['PPRCZ', 'PIMGZ', 'PSTTS', 'PDTLS', 'PBEST', 'RTYPE']:
        if processor.get(deleted_key) is not None:
            del processor[deleted_key]

#save processed processor
with open('./data/processed/processor.json', 'w') as f:
    json.dump(processed_processor, f, indent=2)

[590000, -5000]


### Socket

In [41]:
for socket in list_socket:
    for deleted_key in ['PPRCZ', 'PIMGZ', 'PSTTS', 'PDTLS', 'PBEST']:
        if socket.get(deleted_key) is not None:
            del socket[deleted_key]
with open('./data/processed/socket.json', 'w') as f:
    json.dump(list_socket, f, indent=2)

### Other

In [59]:
for part in ['casing','gpu','hdd','keyboard','monitor','motherboard','mouse','psu','ram','ssd']:
    with open(f'./data/raw/{part}.json') as f:
        item_json = json.load(f)

    list_item = processors['result']
    #remove item if item["PLINK"] contains bundle
    list_item = [item for item in item_json["result"] if "bundle" not in item["PLINK"].lower()]

    

    for item in list_item:
        item['price'] = item['PPRCZ'][0] if item.get('PPRCZ') is not None else None
        item['image'] = item['PIMGZ'][0] if item.get('PIMGZ') is not None else None


    #remove duplicated item
    list_item = list({v['PCODE']:v for v in list_item}.values())

    if part in ['hdd','ssd','ram','gpu']:
        #add size by finding xxGB in PLINK
        for item in list_item:
            capacity = re.findall(r'\d+GB|\d+MB|\d+TB', item['PLINK'])
            if len(capacity) > 0:
                item['size'] = capacity[0]
            else:
                item['size'] = None
    
    if part in ['gpu']:
        for item in list_item:
            #add brand by finding Nvidia or AMD in PLINK
            item['brand'] = 'nvidia' if ('nvidia' in item['PLINK'].lower() or
                                        'geforce' in item['PLINK'].lower() or
                                        'gt' in item['PLINK'].lower() or
                                        'rtx' in item['PLINK'].lower() or
                                        'quadro' in item['PLINK'].lower())    else 'amd' if ('amd' in item['PLINK'].lower() or
                                                        'radeon' in item['PLINK'].lower()
                                                        or 'rx' in item['PLINK'].lower())  else 'intel'

    for item in list_item:
        for deleted_key in ['PPRCZ', 'PIMGZ', 'PSTTS', 'PDTLS', 'PBEST', 'RTYPE']:
            if item.get(deleted_key) is not None:
                del item[deleted_key]

    #save processed item
    with open(f'./data/processed/{part}.json', 'w') as f:
        json.dump(list_item, f, indent=2)


### all

In [62]:
rename_key_dict ={
    'PNAME': 'display_name',
    'PLINK': 'name',
    'BNAME': 'brand',
    'PCODE': 'code',
    'RTYPE': 'ram_type',
}

for file in os.listdir('./data/processed'):
    if file.endswith('.json'):
        with open(f'./data/processed/{file}') as f:
            data = json.load(f)
        #rename key
        for item in data:
            for original, replacement in rename_key_dict.items():
                if item.get(original) is not None:
                    item[replacement] = item.pop(original)
    with open(f'./data/processed/{file}', 'w') as f:
        json.dump(data, f, indent=2)

### ram

In [64]:
with open('./data/processed/ram.json') as f:
    ram = json.load(f)
for item in ram:
    ram_type = re.findall(r'DDR\d', item['name'].upper())
    item['ram_type'] = ram_type[0] if len(ram_type) > 0 else None
with open('./data/processed/ram.json', 'w') as f:
    json.dump(ram, f, indent=2)

### socket

In [6]:
with open('./data/processed/socket.json') as f:
    list_socket = json.load(f)
with open('./data/processed/motherboard.json') as f:
    list_motherboard = json.load(f)
processed_motherboard = []

for socket in list_socket:
    motherboard_sockets = [motherboard for motherboard in list_motherboard if socket["name"].upper().replace("-","") in motherboard["name"].upper().replace("-","")]
    print(socket["name"],':',len(motherboard_sockets))
    for motherboard in motherboard_sockets:
        motherboard["socket"] = socket["name"]
    processed_motherboard += motherboard_sockets


with open('./data/processed/motherboard.json', 'w') as f:
    json.dump(processed_motherboard, f, indent=2)

AM5 : 61
sWRX8 : 2
sTRX4 : 3
TR4 : 2
AM4 : 120
FM2 : 3
AM1 : 0
FM2 : 3
AM3 : 0
LGA-1700 : 176
LGA-1200 : 61
LGA-2066 : 5
LGA-3647 : 0
LGA-1151 : 35
LGA-1150 : 14
LGA-2011 : 0
LGA-1155 : 9
LGA-775 : 3
